## 2 Predictor Variables

What characteristics of US counties might be predictive of drug-related mortality rates? My approach on this matter is heavily influenced by the recent work of Ann Case and Angus Deaton, two Princeton economists who became interested in opioid deaths in 2015 after noticing the [trend-busting decline](http://www.pnas.org/content/112/49/15078) in life expectency for middle-aged white Americans. In a spring 2017 [Brookings Institute article](https://www.brookings.edu/past-bpea-editions/), they follow up on this trend through exploratory research. They develop a broad hypothesis that increases in "deaths of despair" (defined as drug overdose, alcohol-related deaths, and suicide) are one component of a larger trend:

> We propose a preliminary but plausible story in which cumulative disadvantage from one birth cohort to the next—in the labor market, in marriage and child outcomes, and in health—is triggered by progressively worsening labor market opportunities at the time of entry for whites with low levels of
education. This account, which fts much of the data, has the profoundly negative implication that policies—even ones that successfully improve earnings and jobs, or redistribute income—will take many years to reverse the increase in mortality and morbidity, and that those in midlife now are likely to do worse in old age than the current elderly.

In a [Washington Post](https://www.washingtonpost.com/opinions/the-truth-about-deaths-of-despair/2017/09/12/15aa6212-8459-11e7-902a-2a9f2d808496_story.html?utm_term=.c3c7ce910d18) opinion piece they state:

> Opioids are like guns handed out in a suicide ward; they have certainly made the total epidemic much worse, but they are not the cause of the underlying depression. We suspect that deaths of despair among those without a university degree are primarily the result of a 40-year stagnation of median real wages and a long-term decline in the number of well-paying jobs for those without a bachelor's degree. Falling labor force participation, sluggish wage growth, and associated dysfunctional marriage and child-rearing patterns have undermined the meaning of working people's lives as well.

From the work of Case and Deaton we can develop a preliminary list of potential correlates with drug-related mortality:


#### Demographics
 - Age 
 - Race 
 - Hispanic ethnicity 
 - Sex 
 - Marital status
 - Children's living arrangements
 
#### Economic indicators
 - Income 
 - Poverty rates
 - Public assistance
 - Unemployment
 - Labor force participation
 - Education
 - Health insurance
 
Many of the above indicators can be derived from the American Community Survey from the US Census Bureau.

## 2.1 Extract and process county-level census data for use with drug-related mortality data.

This code uses the Census Bureau API to extract data from the 5-year [American Community Survey](https://www.census.gov/programs-surveys/acs/about.html) (ACS). The data is representative of a 5-year period, in this case 2011-2015. There are also 3- and 1- year data files available, but those datasets do not contain estimates for places with fewer than 20,000 or 65,000 residents respectively, which would create a problem with missing data. 

County level data on [urban/rural](https://www.cdc.gov/nchs/data_access/urban_rural.htm) status from NCHS is also incorporated via URL. The final product is a csv file that can later be joined with the drug mortality data created in the first notebook. Due to division of county boundaries over time, the NCHS drug mortality data contains fewer counties than the 2015 ACS. The census data for these counties is aggregated to align with the drug mortality data.

Note that for each census table processed below I use the variable 'totpop' to denote the universe for that table, which is used as a denominator. The instance of totpop that is saved in the final dataset reflects the estimated total population of the county.

In [1]:
# set up CensusData
import censusdata
import pandas as pd
from sodapy import Socrata

pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 2)

In [2]:
# Just an example of how to search for tables. This will produce long output.
#censusdata.search('acs1', '2015', 'label', 'housing')

In [3]:
# How to get field names and descriptions from a specific table. Also produces long output.
# censusdata.printtable(censusdata.censustable('acs5', '2015', 'B15002'))

In [4]:
# function to download and convert ACS data
def get_census(census_vars, year):
    """ Takes list of census variables and year as input, downloads them from ACS5, cleans up index data"""
    census_raw = censusdata.download('acs5', year, censusdata.censusgeo([('county', '*')]), census_vars) 
    census_raw.reset_index(inplace = True)
    
    census_raw['county'] = census_raw['index'].astype(str).str.split(',', 1).str[0] 
    census_raw['summary'] = census_raw['index'].astype(str).str.split(',', 1).str[1]
    census_raw['state'] = census_raw['summary'].astype(str).str.split(':', 1).str[0]
    census_raw['messy_fips'] = census_raw['index'].astype(str).str.split(',', 2).str[2]

    census_raw['state_fips_text'] = census_raw['messy_fips'].astype(str).str.split('>', 1).str[0]
    census_raw['state_fips'] = census_raw['state_fips_text'].astype(str).str.split(':', 1).str[1]

    census_raw['county_fips_text'] = census_raw['messy_fips'].astype(str).str.split('>', 1).str[1]
    census_raw['county_fips'] = census_raw['county_fips_text'].astype(str).str.split(':', 1).str[1]

    census_raw['FIPS'] = census_raw['state_fips'] + census_raw['county_fips']

    census_raw = census_raw.drop(columns=['index', 'summary', 'messy_fips', 'state_fips_text', 'state_fips', 'county_fips_text', 'county_fips'])
      
    return census_raw

In [5]:
def fips_recode(x): 
    """Recode Census FIPS codes to match those used by NCHS. Some counties will be aggregated."""
    if x == "02198": 
        return "02201"
    elif x == "02158": 
        return "02270"
    elif x == "02275": 
        return "02280"
    elif x == "02195": 
        return "02280"
    elif x == "02230": 
        return "02232"
    elif x == "02105": 
        return "02232" 
    elif x == "08059": 
        return "08013"
    elif x == "08014": 
        return "08013"
    elif x == "08123": 
        return "08013"
    elif x == "08001": 
        return "08013"
    else:
        return x

### 2.1.1 Table B01001 - sex and age

In [6]:
# Specify vars for table B01001 - totpop, sex, age univariate breakdowns
census_vars = ['B01001_001E', 
'B01001_002E', 
'B01001_003E', 
'B01001_004E', 
'B01001_005E', 
'B01001_006E', 
'B01001_007E', 
'B01001_008E', 
'B01001_009E', 
'B01001_010E', 
'B01001_011E', 
'B01001_012E', 
'B01001_013E', 
'B01001_014E', 
'B01001_015E', 
'B01001_016E', 
'B01001_017E', 
'B01001_018E', 
'B01001_019E', 
'B01001_020E', 
'B01001_021E', 
'B01001_022E', 
'B01001_023E', 
'B01001_024E', 
'B01001_025E', 
'B01001_026E', 
'B01001_027E', 
'B01001_028E', 
'B01001_029E', 
'B01001_030E', 
'B01001_031E', 
'B01001_032E', 
'B01001_033E', 
'B01001_034E', 
'B01001_035E', 
'B01001_036E', 
'B01001_037E', 
'B01001_038E', 
'B01001_039E', 
'B01001_040E', 
'B01001_041E', 
'B01001_042E', 
'B01001_043E', 
'B01001_044E', 
'B01001_045E', 
'B01001_046E', 
'B01001_047E', 
'B01001_048E', 
'B01001_049E']

In [7]:
C = get_census(census_vars, 2015)

In [8]:
# Recode FIPS to match aggregated NCHS data, then sum numeric data
C['FIPS_NCHS'] = C['FIPS'].apply(fips_recode) 
C = C.groupby('FIPS_NCHS').sum()
C.reset_index(inplace = True)

In [9]:
#C.head()

In [10]:
# Create age and sex percent columns
C['totpop'] = C.B01001_001E

# Age breakdown
C['age_under_5_p'] = (C.B01001_003E + C.B01001_027E)/C.totpop *100
C['age_5_9_p'] = (C.B01001_004E + C.B01001_028E)/C.totpop *100
C['age_10_14_p'] = (C.B01001_005E + C.B01001_029E)/C.totpop *100
C['age_15_17_p'] = (C.B01001_006E + C.B01001_030E)/C.totpop *100
C['age_18_19_p'] = (C.B01001_007E + C.B01001_031E)/C.totpop *100
C['age_20_p'] = (C.B01001_008E + C.B01001_032E)/C.totpop *100
C['age_21_p'] = (C.B01001_009E + C.B01001_033E)/C.totpop *100
C['age_22_24_p'] = (C.B01001_010E + C.B01001_034E)/C.totpop *100
C['age_25_29_p'] = (C.B01001_011E + C.B01001_035E)/C.totpop *100
C['age_30_34_p'] = (C.B01001_012E + C.B01001_036E)/C.totpop *100
C['age_35_39_p'] = (C.B01001_013E + C.B01001_037E)/C.totpop *100
C['age_40_44_p'] = (C.B01001_014E + C.B01001_038E)/C.totpop *100
C['age_45_49_p'] = (C.B01001_015E + C.B01001_039E)/C.totpop *100
C['age_50_54_p'] = (C.B01001_016E + C.B01001_040E)/C.totpop *100
C['age_55_59_p'] = (C.B01001_017E + C.B01001_041E)/C.totpop *100
C['age_60_61_p'] = (C.B01001_018E + C.B01001_042E)/C.totpop *100
C['age_62_64_p'] = (C.B01001_019E + C.B01001_043E)/C.totpop *100
C['age_65_66_p'] = (C.B01001_020E + C.B01001_044E)/C.totpop *100
C['age_67_69_p'] = (C.B01001_021E + C.B01001_045E)/C.totpop *100
C['age_70_74_p'] = (C.B01001_022E + C.B01001_046E)/C.totpop *100
C['age_75_79_p'] = (C.B01001_023E + C.B01001_047E)/C.totpop *100
C['age_80_84_p'] = (C.B01001_024E + C.B01001_048E)/C.totpop *100
C['age_85_over_p'] = (C.B01001_025E + C.B01001_049E)/C.totpop *100

# Sex breakdown
C['sex_male_p'] = C.B01001_002E/C.totpop *100
C['sex_female_p'] = C.B01001_026E/C.totpop *100

C.head()


,FIPS_NCHS,B01001_001E,B01001_002E,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,...,age_60_61_p,age_62_64_p,age_65_66_p,age_67_69_p,age_70_74_p,age_75_79_p,age_80_84_p,age_85_over_p,sex_male_p,sex_female_p
0,01001,55221,26745,1686,1923,2154,1272,764,388,376,...,2.12,2.77,1.89,2.68,3.69,2.46,1.73,1.10,48.43,51.57
1,01003,195121,95314,5261,6366,6675,3887,2209,736,958,...,2.71,4.48,2.68,3.68,4.42,3.14,2.21,2.00,48.85,51.15
2,01005,26932,14497,758,891,844,503,288,187,200,...,2.52,3.50,2.32,3.29,4.11,2.75,2.04,1.55,53.83,46.17
3,01007,22604,12073,540,678,750,564,417,113,129,...,2.63,3.39,1.70,3.10,3.99,2.50,1.91,1.20,53.41,46.59
4,01009,57710,28512,1824,1628,2209,1267,830,266,383,...,2.41,4.00,2.97,3.09,4.00,2.81,2.16,1.45,49.41,50.59


In [11]:
#Create data frame with only needed columns
acs5_2015_B01001 = C[['FIPS_NCHS', 'totpop', 
                    'age_under_5_p', 
'age_5_9_p', 
'age_10_14_p', 
'age_15_17_p', 
'age_18_19_p', 
'age_20_p', 
'age_21_p', 
'age_22_24_p', 
'age_25_29_p', 
'age_30_34_p', 
'age_35_39_p', 
'age_40_44_p', 
'age_45_49_p', 
'age_50_54_p', 
'age_55_59_p', 
'age_60_61_p', 
'age_62_64_p', 
'age_65_66_p', 
'age_67_69_p', 
'age_70_74_p', 
'age_75_79_p', 
'age_80_84_p', 
'age_85_over_p',
'sex_male_p', 
'sex_female_p'
]]


#acs5_2015_B01001.head()

In [12]:
C.head()

,FIPS_NCHS,B01001_001E,B01001_002E,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,...,age_60_61_p,age_62_64_p,age_65_66_p,age_67_69_p,age_70_74_p,age_75_79_p,age_80_84_p,age_85_over_p,sex_male_p,sex_female_p
0,01001,55221,26745,1686,1923,2154,1272,764,388,376,...,2.12,2.77,1.89,2.68,3.69,2.46,1.73,1.10,48.43,51.57
1,01003,195121,95314,5261,6366,6675,3887,2209,736,958,...,2.71,4.48,2.68,3.68,4.42,3.14,2.21,2.00,48.85,51.15
2,01005,26932,14497,758,891,844,503,288,187,200,...,2.52,3.50,2.32,3.29,4.11,2.75,2.04,1.55,53.83,46.17
3,01007,22604,12073,540,678,750,564,417,113,129,...,2.63,3.39,1.70,3.10,3.99,2.50,1.91,1.20,53.41,46.59
4,01009,57710,28512,1824,1628,2209,1267,830,266,383,...,2.41,4.00,2.97,3.09,4.00,2.81,2.16,1.45,49.41,50.59


### 2.1.2 Table B02001 - race

In [13]:
# Specify vars for table B02001 - totpop, race univariate breakdowns
census_vars = ['B02001_001E', 
'B02001_002E', 
'B02001_003E', 
'B02001_004E', 
'B02001_005E', 
'B02001_006E', 
'B02001_007E', 
'B02001_008E' 
]

In [14]:
C = get_census(census_vars, 2015)
#C.head()

In [15]:
# Recode FIPS to match aggregated NCHS data, then sum numeric data

C['FIPS_NCHS'] = C['FIPS'].apply(fips_recode) 
C = C.groupby('FIPS_NCHS').sum()
C.reset_index(inplace = True)

#C.info()

In [16]:
# Create race percent columns
C['totpop'] = C.B02001_001E

# Race breakdown
C['race_white_p'] = (C.B02001_002E)/C.totpop *100
C['race_black_p'] = (C.B02001_003E)/C.totpop *100
C['race_native_amer_p'] = (C.B02001_004E)/C.totpop *100
C['race_asian_p'] = (C.B02001_005E)/C.totpop *100
C['race_pac_islander_p'] = (C.B02001_006E)/C.totpop *100
C['race_other_p'] = (C.B02001_007E)/C.totpop *100
C['race_multi_p'] = (C.B02001_008E)/C.totpop *100

acs5_2015_B02001 = C[['FIPS_NCHS',  
'race_white_p', 
'race_black_p', 
'race_native_amer_p', 
'race_asian_p', 
'race_pac_islander_p', 
'race_other_p', 
'race_multi_p',]]
                      

In [17]:
#acs5_2015_B02001.head()

### 2.1.3 Table B09005 - living arrangements of children

In [18]:
# Specify vars for table B09005 - living arrangements of children
census_vars = ['B09005_001E', 
'B09005_002E', 
'B09005_003E', 
'B09005_004E', 
'B09005_005E', 
'B09005_006E']


In [19]:
C = get_census(census_vars, 2015)


In [20]:
# Recode FIPS to match aggregated NCHS data, then sum numeric data

C['FIPS_NCHS'] = C['FIPS'].apply(fips_recode) 
C = C.groupby('FIPS_NCHS').sum()
C.reset_index(inplace = True)

#C.info()

In [21]:
# Create child home percent columns
C['totpop'] = (C.B09005_001E)
C['kid_home_fam_p'] = (C.B09005_002E)/C.totpop *100
C['kid_home_fam2_p'] = (C.B09005_003E)/C.totpop *100
C['kid_home_fam1_p'] = (C.B09005_004E + C.B09005_005E)/C.totpop *100
C['kid_home_nonfam_p'] = (C.B09005_006E)/C.totpop *100

acs5_2015_B09005 = C[['FIPS_NCHS',  
'kid_home_fam_p', 
'kid_home_fam2_p', 
'kid_home_fam1_p', 
'kid_home_nonfam_p'
]]


### 2.1.4 Table B09021 - living arrangements of adults

In [22]:
# Specify vars for table B09021 - living arrangements of adults
census_vars = ['B09021_001E', 
'B09021_002E', 
'B09021_003E', 
'B09021_004E', 
'B09021_005E', 
'B09021_006E', 
'B09021_007E']

In [23]:
C = get_census(census_vars, 2015)

In [24]:
# Recode FIPS to match aggregated NCHS data, then sum numeric data

C['FIPS_NCHS'] = C['FIPS'].apply(fips_recode) 
C = C.groupby('FIPS_NCHS').sum()
C.reset_index(inplace = True)

#C.info()

In [25]:
# Create adult home percent columns
C['totpop'] = (C.B09021_001E)
C['adult_home_alone_p'] = (C.B09021_002E)/C.totpop *100
C['adult_home_fam_p'] = (C.B09021_003E + C.B09021_004E + C.B09021_005E + C.B09021_006E)/C.totpop *100
C['adult_home_nonfam_p'] = (C.B09021_007E)/C.totpop *100

acs5_2015_B09021 = C[['FIPS_NCHS',  
'adult_home_alone_p',
'adult_home_fam_p', 
'adult_home_nonfam_p'
]]


### 2.1.5 Table B12001 - marital status

In [26]:
# Specify vars for table B12001 - marital status
census_vars = ['B12001_001E', 
'B12001_002E', 
'B12001_003E', 
'B12001_004E', 
'B12001_005E', 
'B12001_006E', 
'B12001_007E', 
'B12001_008E', 
'B12001_009E', 
'B12001_010E', 
'B12001_011E', 
'B12001_012E', 
'B12001_013E', 
'B12001_014E', 
'B12001_015E', 
'B12001_016E', 
'B12001_017E', 
'B12001_018E', 
'B12001_019E']

In [27]:
C = get_census(census_vars, 2015)

In [28]:
# Recode FIPS to match aggregated NCHS data, then sum numeric data

C['FIPS_NCHS'] = C['FIPS'].apply(fips_recode) 
C = C.groupby('FIPS_NCHS').sum()
C.reset_index(inplace = True)

#C.info()

In [29]:
# Create marital status percentages
C['totpop'] = C.B12001_001E

C['marry_never_p'] = (C.B12001_003E + C.B12001_012E)/C.totpop *100
C['married_p'] = (C.B12001_005E + C.B12001_014E)/C.totpop *100
C['marry_past_p'] = (C.B12001_007E + C.B12001_016E + C.B12001_008E + C.B12001_017E + C.B12001_009E + \
     C.B12001_018E + C.B12001_010E + C.B12001_019E) / C.totpop *100

acs5_2015_B12001 = C[['FIPS_NCHS',  
'marry_never_p',
'married_p', 
'marry_past_p'
]]

del C

### 2.1.6 Table B15002 - educational attainment

In [30]:
# Specify vars for table B15002 - educational attainment
census_vars = ['B15002_001E', 
'B15002_002E', 
'B15002_003E', 
'B15002_004E', 
'B15002_005E', 
'B15002_006E', 
'B15002_007E', 
'B15002_008E', 
'B15002_009E', 
'B15002_010E', 
'B15002_011E', 
'B15002_012E', 
'B15002_013E', 
'B15002_014E', 
'B15002_015E', 
'B15002_016E', 
'B15002_017E', 
'B15002_018E', 
'B15002_019E', 
'B15002_020E', 
'B15002_021E', 
'B15002_022E', 
'B15002_023E', 
'B15002_024E', 
'B15002_025E', 
'B15002_026E', 
'B15002_027E', 
'B15002_028E', 
'B15002_029E', 
'B15002_030E', 
'B15002_031E', 
'B15002_032E', 
'B15002_033E', 
'B15002_034E', 
'B15002_035E']

In [31]:
C = get_census(census_vars, 2015)

In [32]:
# Recode FIPS to match aggregated NCHS data, then sum numeric data

C['FIPS_NCHS'] = C['FIPS'].apply(fips_recode) 
C = C.groupby('FIPS_NCHS').sum()
C.reset_index(inplace = True)

#C.info()

In [33]:
# Create education percentages
C['totpop'] = C.B15002_001E

C['ed_at_HS_m'] = (C.B15002_003E + C.B15002_004E + C.B15002_005E + C.B15002_006E + C.B15002_007E + C.B15002_008E + C.B15002_009E + C.B15002_010E + C.B15002_011E)
C['ed_at_some_coll_m'] = (C.B15002_012E + C.B15002_013E + C.B15002_014E)
C['ed_at_coll_grad_m'] = (C.B15002_015E + C.B15002_016E + C.B15002_017E + C.B15002_018E)

C['ed_at_HS_f'] = (C.B15002_020E + C.B15002_021E + C.B15002_022E + C.B15002_023E + C.B15002_024E + C.B15002_025E + C.B15002_026E + C.B15002_027E + C.B15002_028E)
C['ed_at_some_coll_f'] = (C.B15002_029E + C.B15002_030E + C.B15002_031E)
C['ed_at_coll_grad_f'] = (C.B15002_032E + C.B15002_033E + C.B15002_034E + C.B15002_035E)

C['ed_at_HS_p'] = (C.ed_at_HS_m + C.ed_at_HS_f)/C.totpop *100
C['ed_at_some_coll_p'] = (C.ed_at_some_coll_m + C.ed_at_some_coll_f)/C.totpop *100
C['ed_at_coll_grad_p'] = (C.ed_at_coll_grad_m + C.ed_at_coll_grad_f)/C.totpop *100

acs5_2015_B15002 = C[['FIPS_NCHS',
    'ed_at_HS_p', 'ed_at_some_coll_p', 'ed_at_coll_grad_p']]

del C

#acs5_2015_B15002.head()

### 2.1.7 Table B17001 - poverty status

In [34]:
# Specify vars for tableB17001 - poverty status 
# Split due to 50 variable limit

census_vars = ['B17001_001E', 
'B17001_002E', 
'B17001_003E', 
'B17001_004E', 
'B17001_005E', 
'B17001_006E', 
'B17001_007E', 
'B17001_008E', 
'B17001_009E', 
'B17001_010E', 
'B17001_011E', 
'B17001_012E', 
'B17001_013E', 
'B17001_014E', 
'B17001_015E', 
'B17001_016E', 
'B17001_017E', 
'B17001_018E', 
'B17001_019E', 
'B17001_020E', 
'B17001_021E', 
'B17001_022E', 
'B17001_023E', 
'B17001_024E', 
'B17001_025E']

C1 = get_census(census_vars, 2015)



In [35]:
# Recode FIPS to match aggregated NCHS data, then sum numeric data

C1['FIPS_NCHS'] = C1['FIPS'].apply(fips_recode) 
C1 = C1.groupby('FIPS_NCHS').sum()
C1.reset_index(inplace = True)

#C1.info()

In [36]:
# Specify vars for tableB17001 - poverty status 
# Split due to 50 variable limit

census_vars = [               
'B17001_026E', 
'B17001_027E', 
'B17001_028E', 
'B17001_029E', 
'B17001_030E', 
'B17001_031E', 
'B17001_032E', 
'B17001_033E', 
'B17001_034E', 
'B17001_035E', 
'B17001_036E', 
'B17001_037E', 
'B17001_038E', 
'B17001_039E', 
'B17001_040E', 
'B17001_041E', 
'B17001_042E', 
'B17001_043E', 
'B17001_044E', 
'B17001_045E', 
'B17001_046E', 
'B17001_047E', 
'B17001_048E', 
'B17001_049E', 
'B17001_050E', 
'B17001_051E', 
'B17001_052E', 
'B17001_053E', 
'B17001_054E', 
'B17001_055E', 
'B17001_056E', 
'B17001_057E', 
'B17001_058E', 
'B17001_059E']

C2 = get_census(census_vars, 2015)

In [37]:
# Recode FIPS to match aggregated NCHS data, then sum numeric data

C2['FIPS_NCHS'] = C2['FIPS'].apply(fips_recode) 
C2 = C2.groupby('FIPS_NCHS').sum()
C2.reset_index(inplace = True)

#C2.info()

In [38]:
C = C1.merge(C2, on='FIPS_NCHS')

In [39]:
#C.info()

In [40]:
# Create poverty percentages
C['totpop'] = C.B17001_001E

C['pov_p'] = (C.B17001_003E + C.B17001_017E)/C.totpop * 100

C['pov_kid_p'] = (C.B17001_004E + 
C.B17001_005E + 
C.B17001_006E + 
C.B17001_007E + 
C.B17001_008E + 
C.B17001_009E + 
C.B17001_018E + 
C.B17001_019E + 
C.B17001_020E + 
C.B17001_021E + 
C.B17001_022E + 
C.B17001_023E) / (C.B17001_033E + 
C.B17001_034E + 
C.B17001_035E + 
C.B17001_036E + 
C.B17001_037E + 
C.B17001_038E + 
C.B17001_047E + 
C.B17001_048E + 
C.B17001_049E + 
C.B17001_050E + 
C.B17001_051E + 
C.B17001_052E) * 100

C['pov_adult_p'] = (C.B17001_010E + 
C.B17001_011E + 
C.B17001_012E + 
C.B17001_013E + 
C.B17001_014E + 
C.B17001_015E + 
C.B17001_016E + 
C.B17001_024E + 
C.B17001_025E + 
C.B17001_026E + 
C.B17001_027E + 
C.B17001_028E + 
C.B17001_029E + 
C.B17001_030E) / (C.B17001_039E + 
C.B17001_040E + 
C.B17001_041E + 
C.B17001_042E + 
C.B17001_043E + 
C.B17001_044E + 
C.B17001_045E + 
C.B17001_053E + 
C.B17001_054E + 
C.B17001_055E + 
C.B17001_056E + 
C.B17001_057E + 
C.B17001_058E + 
C.B17001_059E) * 100
    
acs5_2015_B17001  = C[['FIPS_NCHS',
    'pov_p', 'pov_kid_p', 'pov_adult_p']]

    

In [41]:
#acs5_2015_b17001.info()

### 2.1.8 Table B18001 - disability status

In [42]:
# Specify vars for tableB17001 - poverty status 
# Split due to 50 variable limit

census_vars = [               
'B18101_001E', 
'B18101_004E', 
'B18101_007E', 
'B18101_010E', 
'B18101_013E', 
'B18101_016E', 
'B18101_019E', 
'B18101_023E', 
'B18101_026E', 
'B18101_029E', 
'B18101_032E', 
'B18101_035E', 
'B18101_038E'
]

C = get_census(census_vars, 2015)

In [43]:
# Recode FIPS to match aggregated NCHS data, then sum numeric data

C['FIPS_NCHS'] = C['FIPS'].apply(fips_recode) 
C = C.groupby('FIPS_NCHS').sum()
C.reset_index(inplace = True)

#C.info()

In [44]:
# Create disability percentages
C['totpop'] = C.B18101_001E
C['disability_p'] = (C.B18101_004E +  
C.B18101_007E +  
C.B18101_010E +  
C.B18101_013E +  
C.B18101_016E +  
C.B18101_019E +  
C.B18101_023E +  
C.B18101_026E +  
C.B18101_029E +  
C.B18101_032E +  
C.B18101_035E +  
C.B18101_038E) / C.totpop * 100

acs5_2015_B18101  = C[['FIPS_NCHS',
     'disability_p']]


### 2.1.9 Table B19013, B19083, B19301 - income measures

In [45]:
# continuous measures related to income
#Median household income in the past 12 months (in 2016 inflation-adjusted dollars)
#Gini Index
#Per capita income in the past 12 months (in 2016 inflation-adjusted dollars)

#### For some reason the API is not working for these variable for 2015 5-year estimates. Other years seem to work.
#census_vars = ['B19013_001E', 'B19083_001E', 'B19301_001E']
# C = get_census(census_vars, 2015)
#C['med_hh_inc'] = C.B19013_001E
#C['gini_index'] = C.B19083_001E 
#C['inc_per_capita'] = C.B19301_001E
#acs5_2015_income  = C[['FIPS_NCHS', 'med_hh_inc', 'gini_index', 'inc_per_capita']]


# I downloaded these from American Fact Finder, since the API couldn't get them.
mhh_raw = pd.read_csv('./census/ACS_15_5YR_B19013.csv', encoding = "ISO-8859-1")
gini_raw = pd.read_csv('./census/ACS_15_5YR_B19083.csv', encoding = "ISO-8859-1")
inc_cap_raw = pd.read_csv('./census/ACS_15_5YR_B19301.csv', encoding = "ISO-8859-1")

acs5_2015_income  = mhh_raw.merge(gini_raw, on = ['FIPS', 'county', 'state']).merge(inc_cap_raw, on = ['FIPS', 'county', 'state'])

# Convert FIPS to string and add leading 0 where needed
acs5_2015_income['FIPS'] = acs5_2015_income.FIPS.astype(str)
condition = acs5_2015_income['FIPS'].str.len() == 4
column_name = 'FIPS'
acs5_2015_income.loc[condition, column_name] = "0" + acs5_2015_income['FIPS']


In [46]:
# Recode FIPS to match aggregated NCHS data, then obtain mean for numeric data

acs5_2015_income['FIPS_NCHS'] = acs5_2015_income['FIPS'].apply(fips_recode) 
acs5_2015_income = acs5_2015_income.groupby('FIPS_NCHS').mean()
acs5_2015_income.reset_index(inplace = True)


In [47]:
#acs5_2015_income.head()

### 2.1.10 Table B21001 - Veteran status

In [48]:
census_vars = [
'B21001_001E', 
'B21001_002E', 
'B21001_005E', 
'B21001_008E', 
'B21001_011E', 
'B21001_014E', 
'B21001_017E', 
'B21001_020E', 
'B21001_023E', 
'B21001_026E', 
'B21001_029E', 
'B21001_032E', 
'B21001_035E', 
'B21001_038E'
]

C = get_census(census_vars, 2015)

In [49]:
# Recode FIPS to match aggregated NCHS data, then sum numeric data

C['FIPS_NCHS'] = C['FIPS'].apply(fips_recode) 
C = C.groupby('FIPS_NCHS').sum()
C.reset_index(inplace = True)

#C.info()

In [50]:
C['totpop'] = C.B21001_001E
C['vet_p'] = (C.B21001_002E +  
C.B21001_005E +  
C.B21001_008E +  
C.B21001_011E +  
C.B21001_014E +  
C.B21001_017E +  
C.B21001_020E +  
C.B21001_023E +  
C.B21001_026E +  
C.B21001_029E +  
C.B21001_032E +  
C.B21001_035E +  
C.B21001_038E) / C.totpop * 100

acs5_2015_B21001  = C[['FIPS_NCHS',
     'vet_p']]


In [51]:
#acs5_2015_B21001.head()

### 2.1.11 Table B19's - income sources

In [52]:
census_vars = [
'B19051_001E', 
'B19051_002E', 
'B19055_002E',  
'B19056_002E', 
'B19058_002E'
]

C = get_census(census_vars, 2015)

In [53]:
# Recode FIPS to match aggregated NCHS data, then sum numeric data

C['FIPS_NCHS'] = C['FIPS'].apply(fips_recode) 
C = C.groupby('FIPS_NCHS').sum()
C.reset_index(inplace = True)

#C.info()

In [54]:
C['totpop'] = C.B19051_001E

C['earn_inc_p'] = C.B19051_002E / C.totpop * 100
C['ss_inc_p'] = C.B19055_002E / C.totpop * 100
C['ssi_inc_p'] = C.B19056_002E / C.totpop * 100
C['pub_inc_p'] = C.B19058_002E / C.totpop * 100

acs5_2015_income_source  = C[['FIPS_NCHS',
     'earn_inc_p', 'ss_inc_p', 'ssi_inc_p', 'pub_inc_p']]

### 2.1.12 Table B03001 - Hispanic ethnicity

In [55]:
census_vars = [
'B03001_001E', 
'B03001_003E'
]

C = get_census(census_vars, 2015)

In [56]:
# Recode FIPS to match aggregated NCHS data, then sum numeric data

C['FIPS_NCHS'] = C['FIPS'].apply(fips_recode) 
C = C.groupby('FIPS_NCHS').sum()
C.reset_index(inplace = True)

#C.info()

In [57]:
# percent Hispanic

C['hisp_p'] = C.B03001_003E / C.B03001_001E * 100

acs5_2015_B03001  = C[['FIPS_NCHS',
     'hisp_p']]

### 2.1.13 Table B27011  - Employment and Insurance status

In [58]:
# Specify vars for tableB27011 - employment and insurance
census_vars = ['B27011_001E', 
'B27011_002E', 
'B27011_003E', 
'B27011_004E', 
'B27011_005E', 
'B27011_006E', 
'B27011_007E', 
'B27011_008E', 
'B27011_009E', 
'B27011_010E', 
'B27011_011E', 
'B27011_012E', 
'B27011_013E', 
'B27011_014E', 
'B27011_015E', 
'B27011_016E', 
'B27011_017E', 
'B27011_018E', 
'B27011_019E', 
'B27011_020E', 
'B27011_021E', 
'B27011_022E', 
'B27011_023E', 
'B27011_024E', 
'B27011_025E',
'B27011_026E', 
'B27011_027E', 
'B27011_028E', 
'B27011_029E', 
'B27011_030E', 
'B27011_031E', 
'B27011_032E', 
'B27011_033E', 
'B27011_034E', 
'B27011_035E'
]

C = get_census(census_vars, 2015)

In [59]:
# Recode FIPS to match aggregated NCHS data, then sum numeric data

C['FIPS_NCHS'] = C['FIPS'].apply(fips_recode) 
C = C.groupby('FIPS_NCHS').sum()
C.reset_index(inplace = True)

#C.info()

In [60]:
# Create measures for employment and insurance

C['totpop'] = C.B27011_001E
C['labor_force_p'] = C.B27011_002E / C.totpop * 100
C['unemp_p'] = C.B27011_014E / C.B27011_002E * 100

# private insurance, adults 18-64
C['ins_priv_18_64_p'] = (C.B27011_006E +  
C.B27011_017E +  
C.B27011_028E) / (C.B27011_004E +  
C.B27011_015E +  
C.B27011_026E) * 100

C['ins_priv_65_up_p'] = (C.B27011_011E +  
C.B27011_022E + 
C.B27011_033E) / (C.B27011_009E +  
C.B27011_020E +  
C.B27011_031E) * 100

C['ins_pub_18_64_p'] = (C.B27011_007E +  
C.B27011_018E +  
C.B27011_029E) / (C.B27011_004E +  
C.B27011_015E +  
C.B27011_026E) * 100

C['ins_pub_65_up_p'] = (C.B27011_012E +  
C.B27011_023E + 
C.B27011_034E) / (C.B27011_009E +  
C.B27011_020E +  
C.B27011_031E) * 100

C['ins_none_18_64_p'] = (C.B27011_008E +  
C.B27011_019E +  
C.B27011_030E) / (C.B27011_004E +  
C.B27011_015E +  
C.B27011_026E) * 100

C['ins_none_65_up_p'] = (C.B27011_013E +  
C.B27011_024E + 
C.B27011_035E) / (C.B27011_009E +  
C.B27011_020E +  
C.B27011_031E) * 100

acs5_2015_B27011 = C[['FIPS_NCHS',
     'labor_force_p', 'unemp_p', 'ins_priv_18_64_p', 'ins_priv_65_up_p', 
     'ins_pub_18_64_p', 'ins_pub_65_up_p', 'ins_none_18_64_p', 'ins_none_65_up_p' ]]


### 2.2 Urban-rural (from NCHS)
https://www.cdc.gov/nchs/data_access/urban_rural.htm

In [61]:
urban = pd.read_excel('https://www.cdc.gov/nchs/data/data_acces_files/NCHSURCodes2013.xlsx', skiprows = 1, names = ['FIPS', 'a', 'b', 'c', 'd', 'e', 'urban_2013', 'urban_2006', 'urban_1990'])
# the 1990 data has a bunch of missing rows and probably won't get used anyway
urban = urban.drop(columns=['a', 'b', 'c', 'd', 'e', 'urban_1990'])

# Convert FIPS to string and add leading 0 where needed
urban['FIPS'] = urban.FIPS.astype(str)
condition = urban['FIPS'].str.len() == 4
column_name = 'FIPS'
urban.loc[condition, column_name] = "0" + urban['FIPS']

urban['FIPS_NCHS'] = urban['FIPS'].apply(fips_recode) 

In [62]:
# For CO counties that will be aggregated, set 2013 urban category to 2 
urban.loc[urban['FIPS_NCHS'] == "08013", 'urban_2013'] = 2

#urban[urban.FIPS_NCHS == "08013"] 

In [63]:
# For AK counties that will be aggregated, set 2013 urban category to 6
urban.loc[urban['FIPS_NCHS'] == "02232", 'urban_2013'] = 6
urban.loc[urban['FIPS_NCHS'] == "02280", 'urban_2013'] = 6
urban.loc[urban['FIPS_NCHS'] == "02270", 'urban_2013'] = 6
urban.loc[urban['FIPS_NCHS'] == "02201", 'urban_2013'] = 6
urban.loc[urban['FIPS_NCHS'] == "02232", 'urban_2013'] = 6

#urban[urban.FIPS_NCHS == "02232"] 

In [64]:
# Aggregate urban categories - use max for counties we have just set to constant values
urban = urban.groupby('FIPS_NCHS').max()
urban.reset_index(inplace = True)

urban.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3138 entries, 0 to 3137
Data columns (total 4 columns):
FIPS_NCHS     3138 non-null object
FIPS          3138 non-null object
urban_2013    3138 non-null int64
urban_2006    3138 non-null int64
dtypes: int64(2), object(2)
memory usage: 98.1+ KB


## 2.3 Combine data frames

In [65]:
# Data Frames to combine:
# acs5_2015_B01001
# acs5_2015_B02001
# acs5_2015_B09005
# acs5_2015_B09021
# acs5_2015_B12001
# acs5_2015_B15002
# acs5_2015_B17001 
# acs5_2015_B18101
# acs5_2015_income
# acs5_2015_B21001
# acs5_2015_income_source
# urban - FIPS only
# acs5_2015_B27011
# acs5_2015_B03001


out1  = acs5_2015_B01001.merge(acs5_2015_B02001, on = 'FIPS_NCHS')
out2  = out1.merge(acs5_2015_B09005, on = 'FIPS_NCHS')
out3  = out2.merge(acs5_2015_B09021, on = 'FIPS_NCHS')
out4  = out3.merge(acs5_2015_B12001, on = 'FIPS_NCHS')
out5  = out4.merge(acs5_2015_B15002, on = 'FIPS_NCHS')
out6  = out5.merge(acs5_2015_B17001, on = 'FIPS_NCHS')
out7  = out6.merge(acs5_2015_B18101, on = 'FIPS_NCHS')
out8  = out7.merge(urban, on = 'FIPS_NCHS', how='left').fillna(3)
out9  = out8.merge(acs5_2015_B21001, on = 'FIPS_NCHS')
out10  = out9.merge(acs5_2015_income_source, on = 'FIPS_NCHS')
out11 = out10.merge(acs5_2015_income, on = 'FIPS_NCHS', how='left').fillna("")
out12  = out11.merge(acs5_2015_B27011, on = 'FIPS_NCHS')
out13  = out12.merge(acs5_2015_B03001, on = 'FIPS_NCHS')
# Omit Puerto Rico
county_data_acs5_2015 = out13[out13['FIPS_NCHS'].str[0:2] != "72"]



In [66]:
# Save county-level data for use with drug mortality data
county_data_acs5_2015.to_csv('./data/county_data_acs5_2015.csv')

county_data_acs5_2015.to_pickle('./data/county_data_acs5_2015_df.pickle')

In [67]:
county_data_acs5_2015.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3136 entries, 0 to 3135
Data columns (total 71 columns):
FIPS_NCHS              3136 non-null object
totpop                 3136 non-null int64
age_under_5_p          3136 non-null float64
age_5_9_p              3136 non-null float64
age_10_14_p            3136 non-null float64
age_15_17_p            3136 non-null float64
age_18_19_p            3136 non-null float64
age_20_p               3136 non-null float64
age_21_p               3136 non-null float64
age_22_24_p            3136 non-null float64
age_25_29_p            3136 non-null float64
age_30_34_p            3136 non-null float64
age_35_39_p            3136 non-null float64
age_40_44_p            3136 non-null float64
age_45_49_p            3136 non-null float64
age_50_54_p            3136 non-null float64
age_55_59_p            3136 non-null float64
age_60_61_p            3136 non-null float64
age_62_64_p            3136 non-null float64
age_65_66_p            3136 non-null floa